# Initialization

In [1]:
# Local setup, not needed in cluster
from pyspark.sql import SparkSession
app_name = "standardization_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()

In [ ]:
# Get some data
%%bash
for i in `seq 2000 2016`; do
    wget https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/${i}.csv.gz
    gzip -cd ${i}.csv.gz  | grep -e TMIN -e TMAX | grep ^US > ${i}.csv
done

In [ ]:
# Move to HDFS

In [70]:
# Actual code
from __future__ import print_function
from pyspark.sql import Row
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [32]:
def mkdf(filename):
    df = spark.read.format("csv").option("header", "false").load(filename)
    df = df.selectExpr(
        "_c0 as station",
        "_c1 as date",
        "_c2 as measurement",
        "_c3 as degree",
        "_c4 as m",
        "_c5 as q",
        "_c6 as s",
        "_c7 as time",
    )
    df = df.filter("q is null") # Quality only
    df = df.filter("s==0") # USA only
    df = df.filter("degree is not null")
    df = df.withColumn("degree", df['degree'].cast(IntegerType()))
    return df

In [50]:
df = mkdf('2*.csv')
print(df.show())
print(df.printSchema())

+-----------+--------+-----------+------+----+----+---+----+
|    station|    date|measurement|degree|   m|   q|  s|time|
+-----------+--------+-----------+------+----+----+---+----+
|USC00027281|20080101|       TMAX|   156|null|null|  0|0700|
|USC00027281|20080101|       TMIN|     0|null|null|  0|0700|
|USC00034988|20080101|       TMAX|   133|null|null|  0|0700|
|USC00034988|20080101|       TMIN|   -44|null|null|  0|0700|
|USC00093578|20080101|       TMAX|   178|null|null|  0|0800|
|USC00093578|20080101|       TMIN|    44|null|null|  0|0800|
|USC00109601|20080101|       TMAX|   -89|null|null|  0|1800|
|USC00109601|20080101|       TMIN|  -239|null|null|  0|1800|
|USC00110187|20080101|       TMIN|   -83|null|null|  0|1600|
|USC00419361|20080101|       TMAX|   250|null|null|  0|0800|
|USC00419361|20080101|       TMIN|    44|null|null|  0|0800|
|USC00421168|20080101|       TMAX|     0|null|null|  0|0800|
|USC00421168|20080101|       TMIN|  -150|null|null|  0|0800|
|USC00426568|20080101|  

# Overview

## Numerical standardization across levels of categories
- Pretend these are temperatures on a given location and station indicates the machine id/location (top of furnace #17 for instance)
- S here represents a location, let's pretend those are different parts of a machine and need to be normalized independelty
- The transformation and count take around 1 minute on a macbook over 43 million rows

Note that groupBy statements have terrible scaling properties and should be avoided if at all possible. This method requires a shuffle due to the fact that we need aggregates within each partition window for each data element. 

The following method isn't neccessary if we do not need to work by partition but instead work on the entire DataFrame. If this was the case we would use built-in methods such as MinMaxScaler, StandardScaler, Normalizer (https://spark.apache.org/docs/2.1.1/ml-features.html)

In [52]:
def z_score(c, w):
    return (col(c) - F.mean(c).over(w)) / F.stddev(c).over(w)

def min_max(c, w):
    return (col(c) - F.min(c).over(w)) / (F.max(c).over(w) - F.min(c).over(w))

In [61]:
categories = ['station','s']
target = "degree"
w = Window.partitionBy(*categories)

df_transformed = df.select(
    z_score(target,w).alias("{0}_normalized".format(target)),
    min_max(target,w).alias("{0}_min_max".format(target)),
    *categories
)
df_transformed = df_transformed.withColumn("id", F.monotonically_increasing_id())

In [62]:
df_transformed.cache()
df_transformed.show()
print(df_transformed.count())

+--------------------+-------------------+-----------+---+---+
|   degree_normalized|     degree_min_max|    station|  s| id|
+--------------------+-------------------+-----------+---+---+
| -1.1693525883416795|0.25504587155963304|USC00026180|  0|  0|
|  -1.910720122784184|0.09174311926605505|USC00026180|  0|  1|
| -1.1193727545590386|0.26605504587155965|USC00026180|  0|  2|
| -1.7691105937333684|0.12293577981651377|USC00026180|  0|  3|
|  -1.027743059290864|0.28623853211009176|USC00026180|  0|  4|
| -1.8190904275160094|0.11192660550458716|USC00026180|  0|  5|
| -0.3780052201165342|0.42935779816513764|USC00026180|  0|  6|
| -1.5358713694143784| 0.1743119266055046|USC00026180|  0|  7|
| -0.3780052201165342|0.42935779816513764|USC00026180|  0|  8|
| -1.4025918126606696| 0.2036697247706422|USC00026180|  0|  9|
|-0.42798505389917496|0.41834862385321103|USC00026180|  0| 10|
|  -1.027743059290864|0.28623853211009176|USC00026180|  0| 11|
| -0.7945038349718738| 0.3376146788990826|USC00026180| 

In [64]:
df_transformed.explain()

== Physical Plan ==
InMemoryTableScan [degree_normalized#2534, degree_min_max#2538, station#1176, s#1182, id#2575L]
   +- InMemoryRelation [degree_normalized#2534, degree_min_max#2538, station#1176, s#1182, id#2575L], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *Project [((cast(degree#1196 as double) - _we0#1367) / _we1#1368) AS degree_normalized#1336, (cast((degree#1196 - _we2#1369) as double) / cast((_we3#1370 - _we4#1371) as double)) AS degree_min_max#1340, station#1176, s#1182, monotonically_increasing_id() AS id#1377L]
            +- Window [avg(_w1#1357L) windowspecdefinition(station#1176, s#1182, ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS _we0#1367, stddev_samp(_w2#1358) windowspecdefinition(station#1176, s#1182, ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS _we1#1368, min(degree#1196) windowspecdefinition(station#1176, s#1182, ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS _we2#1369, max(degree#119

## Categorical Normalization

We do fuzzy matching based off sound distance and lexiconical distance, both of which are directly available from Spark's API. Other methods would require a User-Defined Function (UDF) which has terrible scaling properties, but if needed for accuracy could be developed. BDCSCE supports Spark 2.1 instead of the more modern Spark versions 2.3+ which allow for vectorized calculations (Pandas UDFs)

In [97]:
department1 = Row(id='123456', name='Computer Science')
department2 = Row(id='789012', name='Mechanical Physics')
department3 = Row(id='345678', name='Theater and Drama')
department4 = Row(id='901234', name='recreation')
department5 = Row(id='123457', name='Computer tech')
department6 = Row(id='789013', name='Mechanical Physical')
department7 = Row(id='345679', name='Theater and drame')
department8 = Row(id='901235', name='Recreatin')

departments = [
    department1,
    department2,
    department3,
    department4,
    department5,
    department6,
    department7,
    department8
]

In [98]:
cat_df = spark.createDataFrame(departments)

In [99]:
cat_df.show()

+------+-------------------+
|    id|               name|
+------+-------------------+
|123456|   Computer Science|
|789012| Mechanical Physics|
|345678|  Theater and Drama|
|901234|         recreation|
|123457|      Computer tech|
|789013|Mechanical Physical|
|345679|  Theater and drame|
|901235|          Recreatin|
+------+-------------------+



In [100]:
distinct_names = cat_df.selectExpr('name as name1').distinct().cache()
distinct_names.show()

+-------------------+
|              name1|
+-------------------+
|          Recreatin|
| Mechanical Physics|
|         recreation|
|Mechanical Physical|
|  Theater and Drama|
|  Theater and drame|
|   Computer Science|
|      Computer tech|
+-------------------+



In [102]:
distinct_name_pairs = distinct_names.crossJoin(distinct_names.selectExpr('name1 as name2')).cache()
distinct_name_pairs.show()
print(distinct_name_pairs.count())

+------------------+-------------------+
|             name1|              name2|
+------------------+-------------------+
|         Recreatin|          Recreatin|
|         Recreatin| Mechanical Physics|
|         Recreatin|         recreation|
|         Recreatin|Mechanical Physical|
|         Recreatin|  Theater and Drama|
|         Recreatin|  Theater and drame|
|         Recreatin|   Computer Science|
|         Recreatin|      Computer tech|
|Mechanical Physics|          Recreatin|
|Mechanical Physics| Mechanical Physics|
|Mechanical Physics|         recreation|
|Mechanical Physics|Mechanical Physical|
|Mechanical Physics|  Theater and Drama|
|Mechanical Physics|  Theater and drame|
|Mechanical Physics|   Computer Science|
|Mechanical Physics|      Computer tech|
|        recreation|          Recreatin|
|        recreation| Mechanical Physics|
|        recreation|         recreation|
|        recreation|Mechanical Physical|
+------------------+-------------------+
only showing top

In [150]:
distinct_name_pairs_w_metric = distinct_name_pairs.withColumn(
  "name1_name2_levenshtein",
  F.levenshtein(col("name1"), col("name2"))
)

distinct_name_pairs_w_metric = distinct_name_pairs_w_metric.withColumn(
  "name1_soundex",
  soundex(col("name1"))
).withColumn(
  "name2_soundex",
  soundex(col("name2"))
).cache()

distinct_name_pairs_w_metric = distinct_name_pairs_w_metric.withColumn(
  "name1_name2_soundex_equality",
  soundex(col("name1")) == soundex(col("name2"))
).withColumn(
  "name1_name2_levenshtein_equality",
  F.abs(distinct_name_pairs_w_metric.name1_name2_levenshtein) < 2
)

distinct_name_pairs_w_metric = distinct_name_pairs_w_metric.drop(
    'name1_soundx'
)

distinct_name_pairs_w_metric = distinct_name_pairs_w_metric.select('name1','name2','name1_name2_soundex_equality','name1_name2_levenshtein_equality')

In [158]:
soundex_matches = distinct_name_pairs_w_metric.filter(distinct_name_pairs_w_metric['name1_name2_soundex_equality'] == True).select('name1','name2','name1_name2_soundex_equality').cache()
soundex_matches.show()

+-------------------+-------------------+----------------------------+
|              name1|              name2|name1_name2_soundex_equality|
+-------------------+-------------------+----------------------------+
|          Recreatin|          Recreatin|                        true|
|          Recreatin|         recreation|                        true|
| Mechanical Physics| Mechanical Physics|                        true|
| Mechanical Physics|Mechanical Physical|                        true|
|         recreation|          Recreatin|                        true|
|         recreation|         recreation|                        true|
|Mechanical Physical| Mechanical Physics|                        true|
|Mechanical Physical|Mechanical Physical|                        true|
|  Theater and Drama|  Theater and Drama|                        true|
|  Theater and Drama|  Theater and drame|                        true|
|  Theater and drame|  Theater and Drama|                        true|
|  The

In [182]:
soundex_groups = soundex_matches.select("name1", "name2").groupBy('name1').agg(F.collect_set('name2').alias('groups'))
soundex_groups = soundex_groups.select('groups').drop_duplicates()
soundex_groups = soundex_groups.withColumn("groupId", F.monotonically_increasing_id())
soundex_groups.show()

+--------------------+-------------+
|              groups|      groupId|
+--------------------+-------------+
|[Computer Science...|   8589934592|
|[Theater and Dram...| 223338299392|
|[Mechanical Physi...| 369367187456|
|[recreation, Recr...|1675037245440|
+--------------------+-------------+



In [183]:
cat_df.show()

+------+-------------------+
|    id|               name|
+------+-------------------+
|123456|   Computer Science|
|789012| Mechanical Physics|
|345678|  Theater and Drama|
|901234|         recreation|
|123457|      Computer tech|
|789013|Mechanical Physical|
|345679|  Theater and drame|
|901235|          Recreatin|
+------+-------------------+



In [184]:
cat_df.join(soundex_groups, F.expr("array_contains(groups, name)")).show()

+------+-------------------+--------------------+-------------+
|    id|               name|              groups|      groupId|
+------+-------------------+--------------------+-------------+
|123456|   Computer Science|[Computer Science...|   8589934592|
|789012| Mechanical Physics|[Mechanical Physi...| 369367187456|
|345678|  Theater and Drama|[Theater and Dram...| 223338299392|
|901234|         recreation|[recreation, Recr...|1675037245440|
|123457|      Computer tech|[Computer Science...|   8589934592|
|789013|Mechanical Physical|[Mechanical Physi...| 369367187456|
|345679|  Theater and drame|[Theater and Dram...| 223338299392|
|901235|          Recreatin|[recreation, Recr...|1675037245440|
+------+-------------------+--------------------+-------------+

